Create schema

In [0]:
%sql
create schema if not exists bronze;

Create a managed table

In [0]:
%sql
create table if not exists bronze.emp(
  emp_id int,
  emp_name string,
  dept_code string
);

In [0]:
%sql
describe extended bronze.emp;

In [0]:
%sql
insert into bronze.emp values(1, "Alan", "IT");
insert into bronze.emp values(2, "Bethany", "HR");
insert into bronze.emp values(3, "Carla", "FIN");

In [0]:
%sql
select * from bronze.emp;

In [0]:
dbutils.fs.ls("abfss://unity-catalog-storage@dbstorage4azj355ohl4vo.dfs.core.windows.net/3552585333696774/__unitystorage/catalogs/e82e8698-857a-4e14-8568-b93f822acb64/tables/2f2cb6fe-3284-472a-9673-0bc4094ef711") # Unfortunately this does not work for Unity Catalog managed tables

Create an external table

In [0]:
%sql
create external table bronze.emp_ext
using delta
location 'abfss://unity-catalog-storage@dbstorage4azj355ohl4vo.dfs.core.windows.net/3552585333696774/bronze/emp_ext'
as select * from bronze.emp;

In [0]:
%sql
describe extended bronze.emp_ext;

In [0]:
%sql
select * from bronze.emp_ext;

In [0]:
dbutils.fs.ls("abfss://unity-catalog-storage@dbstorage4azj355ohl4vo.dfs.core.windows.net/3552585333696774/bronze/emp_ext")

Differences between managed and external tables

When deleting a managed table, the data itself is deleted as well. When deleting an external table, the data is retained, just the schema for it (and the access data) is deleted.

In [0]:
%sql
-- The following deletes both the metadata and the data (within 30 days unless VACUUM is used)
drop table bronze.emp; 

In [0]:
%sql
-- The following only deletes the metadata and the files would still be visible
drop table bronze.emp_ext;

In [0]:
dbutils.fs.ls("abfss://unity-catalog-storage@dbstorage4azj355ohl4vo.dfs.core.windows.net/3552585333696774/bronze/emp_ext")

In [0]:
%sql
-- The entire external table can be reconstructed after dropping from the files already present
create external table bronze.emp_ext
location "abfss://unity-catalog-storage@dbstorage4azj355ohl4vo.dfs.core.windows.net/3552585333696774/bronze/emp_ext"
-- Not even the schema needs to be specified

In [0]:
%sql
select * from bronze.emp_ext;

Views

In [0]:
%sql
-- create view if not exists bronze.emp_hr_view as select * from bronze.emp where dept_code = 'HR'; This does not work since bronze.emp was dropped
create view if not exists bronze.emp_hr_view as select * from bronze.emp_Ext where dept_code = 'HR';

In [0]:
%sql
select * from bronze.emp_hr_view;

In [0]:
%python
# Keep alive
print("Hello")